In [1]:
# initial imports
import pandas as pd
import numpy as np
from path import Path
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Load data
file_path = Path("Resources/winemag-data_cleaned.csv")
wine_df = pd.read_csv(file_path)
wine_df.head()

,points,price,country,designation,province,region_1,variety,winery,tokens,filtered_tokens,1,2,3,4,5,6,7,8,9,10
0,96.0,235.0,12,20,6,24,2,15,60,36,2701.0,16004.0,23236.0,28401.0,33079.0,39068.0,45330.0,46582.0,63420.0,66669.0
1,96.0,110.0,11,20,26,26,9,15,51,31,11481.0,33140.0,41643.0,46758.0,55709.0,59455.0,69650.0,79653.0,104662.0,106432.0
2,96.0,90.0,12,20,6,26,19,15,47,30,10077.0,26468.0,32151.0,49402.0,61157.0,64289.0,76272.0,88398.0,99749.0,119775.0
3,96.0,65.0,12,22,27,46,12,15,62,43,1546.0,4238.0,13171.0,15324.0,16356.0,25381.0,33140.0,46758.0,56257.0,67144.0
4,95.0,66.0,4,20,31,26,9,15,66,37,1546.0,17161.0,18176.0,31536.0,33140.0,48479.0,49526.0,65848.0,88378.0,90392.0


In [3]:
# Data types
wine_df.dtypes

points             float64
price              float64
country              int64
designation          int64
province             int64
region_1             int64
variety              int64
winery               int64
tokens               int64
filtered_tokens      int64
1                  float64
2                  float64
3                  float64
4                  float64
5                  float64
6                  float64
7                  float64
8                  float64
9                  float64
10                 float64
dtype: object

In [4]:
# find null values
for column in wine_df.columns:
    print(f"Column {column} has {wine_df[column].isnull().sum()} null values")

Column points has 0 null values
Column price has 0 null values
Column country has 0 null values
Column designation has 0 null values
Column province has 0 null values
Column region_1 has 0 null values
Column variety has 0 null values
Column winery has 0 null values
Column tokens has 0 null values
Column filtered_tokens has 0 null values
Column 1 has 0 null values
Column 2 has 0 null values
Column 3 has 0 null values
Column 4 has 0 null values
Column 5 has 0 null values
Column 6 has 0 null values
Column 7 has 0 null values
Column 8 has 0 null values
Column 9 has 0 null values
Column 10 has 0 null values


In [5]:
#Transform features
wine_df["1"] = wine_df["1"] / 100
wine_df["2"] = wine_df["2"] / 100
wine_df["3"] = wine_df["3"] / 100
wine_df["4"] = wine_df["4"] / 100
wine_df["5"] = wine_df["5"] / 100
wine_df["6"] = wine_df["6"] / 100
wine_df["7"] = wine_df["7"] / 100
wine_df["8"] = wine_df["8"] / 100
wine_df["9"] = wine_df["9"] / 100
wine_df["10"] = wine_df["10"] / 100
wine_df.head()

,points,price,country,designation,province,region_1,variety,winery,tokens,filtered_tokens,1,2,3,4,5,6,7,8,9,10
0,96.0,235.0,12,20,6,24,2,15,60,36,27.01,160.04,232.36,284.01,330.79,390.68,453.30,465.82,634.20,666.69
1,96.0,110.0,11,20,26,26,9,15,51,31,114.81,331.40,416.43,467.58,557.09,594.55,696.50,796.53,1046.62,1064.32
2,96.0,90.0,12,20,6,26,19,15,47,30,100.77,264.68,321.51,494.02,611.57,642.89,762.72,883.98,997.49,1197.75
3,96.0,65.0,12,22,27,46,12,15,62,43,15.46,42.38,131.71,153.24,163.56,253.81,331.40,467.58,562.57,671.44
4,95.0,66.0,4,20,31,26,9,15,66,37,15.46,171.61,181.76,315.36,331.40,484.79,495.26,658.48,883.78,903.92


In [6]:
wine_df.insert(1, "points_ranked", '')
wine_df.insert(3, "price_ranked", '')
wine_df.insert(5, "country_name", '')
wine_df.head()

,points,points_ranked,price,price_ranked,country,country_name,designation,province,region_1,variety,...,1,2,3,4,5,6,7,8,9,10
0,96.0,,235.0,,12,,20,6,24,2,...,27.01,160.04,232.36,284.01,330.79,390.68,453.30,465.82,634.20,666.69
1,96.0,,110.0,,11,,20,26,26,9,...,114.81,331.40,416.43,467.58,557.09,594.55,696.50,796.53,1046.62,1064.32
2,96.0,,90.0,,12,,20,6,26,19,...,100.77,264.68,321.51,494.02,611.57,642.89,762.72,883.98,997.49,1197.75
3,96.0,,65.0,,12,,22,27,46,12,...,15.46,42.38,131.71,153.24,163.56,253.81,331.40,467.58,562.57,671.44
4,95.0,,66.0,,4,,20,31,26,9,...,15.46,171.61,181.76,315.36,331.40,484.79,495.26,658.48,883.78,903.92


In [7]:
# points statistics
wine_df['points'].describe()

count    137172.000000
mean         87.789192
std           3.220398
min          80.000000
25%          86.000000
50%          88.000000
75%          90.000000
max         100.000000
Name: points, dtype: float64

In [8]:
# price statistics
wine_df['price'].describe()

count    137172.000000
mean         33.133693
std          36.327952
min           4.000000
25%          16.000000
50%          24.000000
75%          40.000000
max        2300.000000
Name: price, dtype: float64

In [9]:
# excellent = 1 = 100-90
# good = 2 = 89.9 - 88
# average = 3 = 87.9 - 86
# ok = 4 = 85.9 - 80

wine_df.loc[(wine_df['points'] >= 90),'points_ranked'] = 'Excellent'
wine_df.loc[(wine_df['points'] <= 89.9) & (wine_df['points'] >= 88),'points_ranked'] = 'Good'
wine_df.loc[(wine_df['points'] <= 87.9) & (wine_df['points'] >= 86),'points_ranked'] = 'Average'
wine_df.loc[(wine_df['points'] < 85.9),'points_ranked'] = 'ok'
points_ranked_count = wine_df['points_ranked'].value_counts()
print(points_ranked_count)

Excellent    42439
ok           33779
Average      33370
Good         27584
Name: points_ranked, dtype: int64


In [10]:
# top_shelf = 1 = 2300 - 40
# high = 2 = 39.9 - 24
# med = 3 = 23.9 - 16
# low = 4 = 15.9 - 4

wine_df.loc[(wine_df['price'] >= 40),'price_ranked'] = 'Top Shelf'
wine_df.loc[(wine_df['price'] <= 39.9) & (wine_df['price'] >= 24),'price_ranked'] = 'High'
wine_df.loc[(wine_df['price'] <= 23.9) & (wine_df['price'] >= 16),'price_ranked'] = 'Med'
wine_df.loc[(wine_df['price'] < 15.9),'price_ranked'] = 'Low'
price_ranked_count = wine_df['price_ranked'].value_counts()
print(price_ranked_count)

High         36106
Top Shelf    35322
Low          33872
Med          31872
Name: price_ranked, dtype: int64


In [12]:
wine_df.loc[(wine_df['country'] == 0),'country_name'] = 'Argentina'
wine_df.loc[(wine_df['country'] == 1),'country_name'] = 'Austrailia'
wine_df.loc[(wine_df['country'] == 2),'country_name'] = 'Austria'
wine_df.loc[(wine_df['country'] == 3),'country_name'] = 'Chile'
wine_df.loc[(wine_df['country'] == 4),'country_name'] = 'France'
wine_df.loc[(wine_df['country'] == 5),'country_name'] = 'Germany'
wine_df.loc[(wine_df['country'] == 6),'country_name'] = 'Italy'
wine_df.loc[(wine_df['country'] == 7),'country_name'] = 'New Zealand'
wine_df.loc[(wine_df['country'] == 8),'country_name'] = 'Other'
wine_df.loc[(wine_df['country'] == 9),'country_name'] = 'Portugal'
wine_df.loc[(wine_df['country'] == 10),'country_name'] = 'South Africa'
wine_df.loc[(wine_df['country'] == 11),'country_name'] = 'Spain'
wine_df.loc[(wine_df['country'] == 12),'country_name'] = 'US'
country_name_count = wine_df['country_name'].value_counts()
print(country_name_count)

US              62100
Italy           18781
France          14782
Spain            8160
Chile            5766
Argentina        5587
Austrailia       4887
Portugal         4175
New Zealand      3070
Other            2802
Austria          2482
Germany          2345
South Africa     2235
Name: country_name, dtype: int64


In [13]:
wine_df.head(20)

,points,points_ranked,price,price_ranked,country,country_name,designation,province,region_1,variety,...,1,2,3,4,5,6,7,8,9,10
0,96.0,Excellent,235.0,Top Shelf,12,US,20,6,24,2,...,27.01,160.04,232.36,284.01,330.79,390.68,453.30,465.82,634.20,666.69
1,96.0,Excellent,110.0,Top Shelf,11,Spain,20,26,26,9,...,114.81,331.40,416.43,467.58,557.09,594.55,696.50,796.53,1046.62,1064.32
2,96.0,Excellent,90.0,Top Shelf,12,US,20,6,26,19,...,100.77,264.68,321.51,494.02,611.57,642.89,762.72,883.98,997.49,1197.75
3,96.0,Excellent,65.0,Top Shelf,12,US,22,27,46,12,...,15.46,42.38,131.71,153.24,163.56,253.81,331.40,467.58,562.57,671.44
4,95.0,Excellent,66.0,Top Shelf,4,France,20,31,26,9,...,15.46,171.61,181.76,315.36,331.40,484.79,495.26,658.48,883.78,903.92
5,95.0,Excellent,73.0,Top Shelf,11,Spain,20,26,26,9,...,84.08,181.33,331.40,389.11,416.43,421.13,467.58,485.34,510.29,557.09
6,95.0,Excellent,65.0,Top Shelf,11,Spain,20,26,26,9,...,55.61,196.33,264.31,326.80,331.40,431.57,467.58,509.25,775.01,865.77
7,95.0,Excellent,110.0,Top Shelf,11,Spain,20,26,26,9,...,42.35,315.36,331.40,354.18,467.58,510.29,602.34,607.76,670.09,676.62
8,95.0,Excellent,65.0,Top Shelf,12,US,20,27,26,12,...,23.06,31.67,99.02,331.40,523.00,692.99,701.52,720.08,731.56,918.78
9,95.0,Excellent,60.0,Top Shelf,12,US,20,6,39,12,...,33.54,415.02,599.86,816.04,842.21,886.76,903.92,997.49,1000.36,1265.66


In [15]:
# Saving cleaned data
file_path = "Resources/wine_data_tableau.csv"
wine_df.to_csv(file_path, index=False)